In [ ]:
import sys  
sys.path.insert(0, '../')

In [ ]:
import pickle
import pandas as pd
import pprint
from align_and_analyse_ambiguous_trans import analyse_single_sentence_single_perturbed_word, analyse_single_sentence, align_translations, uniquify

In [ ]:
dataset = 'winoMT'
data_root_path = '../data'
src_lang = 'en'
tgt_lang = 'de'

beam = 5
seed = 0
replacement_strategy = 'masking_language_model'
no_of_replacements = 30
unmasking_model = "roberta-base"

if tgt_lang in ['de', 'cs']:
    mask_type = f'MultiplePerSentence_content'
    effecting_words_threshold = 2
    consistence_trans_portion_threshold = 0.95
    uniques_portion_for_noiseORperturbed_threshold = 0.9
    nmt_log_prob_threshold = 0.45
    alignment_tool="Tercom"
elif tgt_lang in ['zh', 'ja']:
    mask_type = f'MultiplePerSentence_allTokens'
    effecting_words_threshold = 4
    consistence_trans_portion_threshold = 0.95
    uniques_portion_for_noiseORperturbed_threshold = 0.8
    nmt_log_prob_threshold = 0.6
    alignment_tool="Tercom"
task = 'trans_word_level_eval'

In [ ]:
QE_method = "nr_effecting_src_words"
with open(f"../analyse_output/winoMT_en2de_qe_wmt21/pred_labels_{QE_method}.pkl", 'rb') as f:
    qe_ok_bad_preds = pickle.load(f)


In [ ]:
perturbed_trans_df = pd.read_pickle(
    "../analyse_output/winoMT_en2de_qe_wmt21/analyse_winoMT_en2de_MultiplePerSentence_content.pkl"
)

In [ ]:
def str_to_list(list_as_str):
    """
    E.g.,
    "[0, 1]" --> [0, 1]
    """
    if list_as_str == '[]':
        return []
    else:
        l = list_as_str[1:-1].split(', ')
        l = [int(x) for x in l]
        return l
    
    

    
true_positive_indices = []

gender_pred = pd.read_csv("../analyse_output/winoMT_en2de_qe_wmt21/gender_pred.csv")
winoMT_data = pd.read_csv(
    "../../mt_gender/data/aggregates/en.txt", 
    sep='\t', header=None, names=['Gender', 'x', 'SRC', 'noun']
)
gender_info = pd.merge(winoMT_data, gender_pred, left_index=True, right_index=True)
# Find out the wrong gender output
gender_info['Correct gender prediction'] = gender_info['Gender'] == gender_info['Predicted gender']

nr_true_positives = 0  # The number of wrongly outputted gender translation that is labeled as BAD
nr_positives = 0  # The total number of wrongly outputted gender translation
for sentence_index in range(winoMT_data.shape[0]):
    true_positive_indices_per_sentence = []
    if not gender_info['Correct gender prediction'].iloc[sentence_index]:
        gender_word_indices = str_to_list(gender_info['Gender words indices'].iloc[sentence_index])
        nr_positives = nr_positives + len(gender_word_indices)

        for gender_word_index in gender_word_indices:
            if qe_ok_bad_preds[sentence_index][gender_word_index] == "BAD":
                nr_true_positives = nr_true_positives + 1
                true_positive_indices_per_sentence.append(gender_word_index)
    true_positive_indices.append(true_positive_indices_per_sentence)


In [ ]:
print("sentences with wrong gender detected")
s = [i for i in range(winoMT_data.shape[0]) if len(true_positive_indices[i])>0]
print(s)

In [ ]:
true_positive_indices[9]

In [ ]:
task = 'trans_word_level_eval'
sentence_idx = 9
word_idx = 4

sentence_df = perturbed_trans_df[perturbed_trans_df['SRC_original_idx'] == sentence_idx]
original_SRC = sentence_df['SRC'].values[0]
original_translation = sentence_df['SRC-Trans'].values[0]
tok_original_translation = sentence_df['tokenized_SRC-Trans'].values[0]
tok_original_SRC = sentence_df['tokenized_SRC'].values[0]
word = tok_original_translation[word_idx]

In [ ]:
print(f"Original SRC sentence:\n{original_SRC}")
print(f"Original trans:\n{original_translation}")
print(f"BAD word: {word}")
print()

pprint.pprint(analyse_single_sentence(
    sentence_df, align_type="trans-only", return_word_index=False,
    consistence_trans_portion_threshold=consistence_trans_portion_threshold,
    uniques_portion_for_noiseORperturbed_threshold=uniques_portion_for_noiseORperturbed_threshold,
    alignment_tool=alignment_tool
)[word])

In [ ]:
original_word = 'chief'


groups_by_perturbed_word = sentence_df.groupby("SRC_masked_index", as_index=False)
original_words = [group_by_perturbed_word.iloc[0]['original_word']
                  for _, group_by_perturbed_word in groups_by_perturbed_word]
groups_by_perturbed_word = [group_by_perturbed_word for _, group_by_perturbed_word in groups_by_perturbed_word]
original_words = list(uniquify(original_words))


sentence_single_perturbed_word_df = groups_by_perturbed_word[original_words.index(original_word)]


pprint.pprint(analyse_single_sentence_single_perturbed_word(
    sentence_single_perturbed_word_df, align_type="trans-only",
    consistence_trans_portion_threshold=consistence_trans_portion_threshold,
    uniques_portion_for_noiseORperturbed_threshold=0.8,
    alignment_tool=alignment_tool
))
tmp_df = align_translations(sentence_single_perturbed_word_df, align_type="trans-only", alignment_tool=alignment_tool)
tmp_df